# Code to scrape player data from Basketball-reference.com # 

**Note:** This 

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import time 

In [2]:
# Here is a function to turn a url address into a beautiful soup object
# url should be a string
# file path should also be a string specifying the path to save the html data
def url_to_soup(url, file_path='', save=0):
    # here we actually access the website
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')

    # save the file
    if save: 
        with open(file_path, 'w') as new_file:
            new_file.write(html)

    # here it's already a local operation
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [3]:
def file_to_df(path):
    with open(path) as file:
        soup = BeautifulSoup(file, 'html.parser')
    df = pd.read_html(str(soup))[0]
    return df

In [4]:
# grab data for Donovan Mitchell from Sports-ref
don_url = 'https://www.basketball-reference.com/players/m/mitchdo01.html'  # contains stats for each of his seasons

don_soup = url_to_soup(don_url, 'Data/Donovan-Mitchell-career-stats')
df_don = pd.read_html(str(don_soup))[0]
df_don[df_don['Season'] == 'Career'].index[0]
df_don = df_don.drop(4)

df_don['Name'] = 'Donovan Mitchell'
df_don.head()

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Name
0,2017-18,21.0,UTA,NBA,SG,79,71,33.4,7.5,17.2,...,0.7,3.1,3.7,3.7,1.5,0.3,2.7,2.7,20.5,Donovan Mitchell
1,2018-19,22.0,UTA,NBA,SG,77,77,33.7,8.6,19.9,...,0.8,3.3,4.1,4.2,1.4,0.4,2.8,2.7,23.8,Donovan Mitchell
2,2019-20,23.0,UTA,NBA,SG,69,69,34.3,8.7,19.4,...,0.8,3.6,4.4,4.3,1.0,0.2,2.7,2.5,24.0,Donovan Mitchell
3,2020-21,24.0,UTA,NBA,SG,45,45,33.6,8.8,20.1,...,1.0,3.4,4.5,5.5,1.0,0.3,3.0,2.2,25.7,Donovan Mitchell


In [48]:
# This function scrapes data from basketball reference
# It appends data to an existing data table for any player that played at least one year after the 79 season

def player_scrape(letter, df_players, delay, first_run=1, names=[], count=0, letter_finished=False):
    url_base = 'https://www.basketball-reference.com'
    url_search = url_base + '/players/' + letter
    if(first_run):
        print('first run')
        # input the url for the page of alphabetical players
        soup_temp = url_to_soup(url_search)
        names = soup_temp.find_all(class_='left')
        # print(names)
        
    # If we were blocked, try to pick up where we left off (remove the names we already covered)
    else:
        print('not first run')
        names = names[count:]
        # print('shortened names: ', names)
    try:   
        # grab the urls for each player and scrape stats
        for count, player in enumerate(names):
            item = str(player.find("a"))
            if '/players/' + letter in item:
                item = item.split('">')
                # print('item: ', item)
                url = item[0].split('href="')[1]
                # print('url: ', url)
                name = item[1].split('</a>')[0]
                # print('name: ', name)

                # Turn url into soup object and extract data table
                soup_player = url_to_soup(url_base + url)
                df_temp = pd.read_html(str(soup_player))[0]

                # drop career stats
                index = df_temp[df_temp['Season'] == 'Career'].index[0]
                df_temp = df_temp.drop(range(index, df_temp.shape[0]))

                #drop data from before a certain year (1979 when 'all modern' stats were tracked)
                # so if any player finished career before 1979, don't append the dataframe to the master list
                final_year = df_temp.iloc[-1, 0].split('-')[0]
                if int(final_year) >= 1979:
                    print('\nPlayers career during/after 1979')
                    
                    # add column with player's name
                    df_temp['Name'] = name
                    print("Found ", name)
                    
                    # Try to extract all star data based off star by name
                    if soup_player.find('tbody').find(class_='sr_star'):
                        print('player was an all star')
                        player_rows = soup_player.find('tbody').find_all('tr')

                        # ignore table header
                        # player_rows = player_rows[1:]
                        #print('\nplayer_rows: ', player_rows, '\n')

                        for season in player_rows:
                            # print(season, '\n')
                            # if the player was an all star, indicate with a 1 in the All Star column for that season
                            year_data = season.find_all('a')
                            if year_data:
                                # print('\n YEAR DATA: \n', year_data, '\n')
                                year = year_data[0].text
                                print(year)
                                
                                # If the player played overseas year is some long note, move on
                                if len(year) < 8:
                                    print('valid year')
                                    year_index = df_temp[df_temp['Season'] == year].index[0]
                                    print(year_index)
                                    if season.find(class_='sr_star'):
                                        print('all star this year, ', year)
                                        df_temp.at[year_index, 'All Star'] = int(1)
                                    else:
                                        df_temp.at[year_index, 'All Star'] = int(0)
                                        print('not an all star this year,', year)
                                
                                else:
                                    df_temp["All Star"] = int(0)

                    # If player was never an all-star
                    else:
                        print('player was never an all star')
                        df_temp["All Star"] = int(0)

                    # append player's data to full table
                    df_players = df_players.append(df_temp)
                        
                    # throw in a delay to avoid getting blocked
                    time.sleep(delay)
            else:
                time.sleep(delay)

            # this is only for development, remove once everything is working 
            # if count == 20:
              #  break 
    except ConnectionResetError:
        # if website blocks us, return data table and where we left off
        print('Blocked')
        time.sleep(3)
        return df_players, names, count, letter_finished
    else:
        print('\n\n Completed!')
        letter_finished = True
        return df_players, names, count, letter_finished

In [24]:
# Function scrapes the number of players
def scrape_name_totals():
    url_base = 'https://www.basketball-reference.com'
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
                'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']
    num = 0
    name_totals = []
    for i in range(len(alphabet)):
        number_soup = url_to_soup(url_base+alphabet[i])
        h2 = number_soup.find_all('h2')
        num = h2[1].text.split(' ')[0]
        # print(int(num))
        name_totals.append(num)
        time.sleep(0)
    return name_totals

In [8]:
# Experiment with a more generic way to scrape data:
# Player data is stored in an alphabetized table sorted by name
# each player has a unique url address with a stats table.('https://www.basketball-reference.com/players/m/mitchdo01.html')
# the /m indicates the first letter of the player's last name followed by his unique url
# We can loop through the letters of the alphabet, and grab the unique url from the last name landing page html

# Define alphabet list, note: no present or historical players have a last name starting with x, so ignore it.
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']
df_players_a = pd.DataFrame()
url_base = 'https://www.basketball-reference.com'
first_url = url_base + '/players/' + alphabet[0]

# for ii in range(len(alphabet)):
df_players_a, names, count_1 = player_scrape('a', df_players_a, delay=1.5)

first run
passed year
Found  Alaa Abdelnaby
player was never an all star
passed year
Found  Kareem Abdul-Jabbar
player was an all star

 YEAR DATA: 
 [<a href="/players/a/abdulka01/gamelog/1970/">1969-70</a>, <a href="/teams/MIL/1970.html">MIL</a>, <a href="/leagues/NBA_1970.html">NBA</a>] 

1969-70
valid year
0
all star this year,  1969-70

 YEAR DATA: 
 [<a href="/players/a/abdulka01/gamelog/1971/">1970-71</a>, <a href="/teams/MIL/1971.html">MIL</a>, <a href="/leagues/NBA_1971.html">NBA</a>] 

1970-71
valid year
1
all star this year,  1970-71

 YEAR DATA: 
 [<a href="/players/a/abdulka01/gamelog/1972/">1971-72</a>, <a href="/teams/MIL/1972.html">MIL</a>, <a href="/leagues/NBA_1972.html">NBA</a>] 

1971-72
valid year
2
all star this year,  1971-72

 YEAR DATA: 
 [<a href="/players/a/abdulka01/gamelog/1973/">1972-73</a>, <a href="/teams/MIL/1973.html">MIL</a>, <a href="/leagues/NBA_1973.html">NBA</a>] 

1972-73
valid year
3
all star this year,  1972-73

 YEAR DATA: 
 [<a href="/players

passed year
Found  Hassan Adams
player was never an all star
passed year
Found  Jaylen Adams
player was never an all star
passed year
Found  Jordan Adams
player was never an all star
passed year
Found  Michael Adams
player was an all star

 YEAR DATA: 
 [<a href="/players/a/adamsmi01/gamelog/1986/">1985-86</a>, <a href="/teams/SAC/1986.html">SAC</a>, <a href="/leagues/NBA_1986.html">NBA</a>] 

1985-86
valid year
0
not an all star this year, 1985-86

 YEAR DATA: 
 [<a href="/players/a/adamsmi01/gamelog/1987/">1986-87</a>, <a href="/teams/WSB/1987.html">WSB</a>, <a href="/leagues/NBA_1987.html">NBA</a>] 

1986-87
valid year
1
not an all star this year, 1986-87

 YEAR DATA: 
 [<a href="/players/a/adamsmi01/gamelog/1988/">1987-88</a>, <a href="/teams/DEN/1988.html">DEN</a>, <a href="/leagues/NBA_1988.html">NBA</a>] 

1987-88
valid year
2
not an all star this year, 1987-88

 YEAR DATA: 
 [<a href="/players/a/adamsmi01/gamelog/1989/">1988-89</a>, <a href="/teams/DEN/1989.html">DEN</a>, <a hr

passed year
Found  Alexis Ajinça
player was never an all star
passed year
Found  Josh Akognon
player was never an all star
passed year
Found  DeVaughn Akoon-Purcell
player was never an all star
passed year
Found  Solomon Alabi
player was never an all star
passed year
Found  Mark Alarie
player was never an all star
passed year
Found  Furkan Aldemir
player was never an all star
passed year
Found  Cole Aldrich
player was never an all star
passed year
Found  LaMarcus Aldridge
player was an all star

 YEAR DATA: 
 [<a href="/players/a/aldrila01/gamelog/2007/">2006-07</a>, <a href="/teams/POR/2007.html">POR</a>, <a href="/leagues/NBA_2007.html">NBA</a>] 

2006-07
valid year
0
not an all star this year, 2006-07

 YEAR DATA: 
 [<a href="/players/a/aldrila01/gamelog/2008/">2007-08</a>, <a href="/teams/POR/2008.html">POR</a>, <a href="/leagues/NBA_2008.html">NBA</a>] 

2007-08
valid year
1
not an all star this year, 2007-08

 YEAR DATA: 
 [<a href="/players/a/aldrila01/gamelog/2009/">2008-09</a>

passed year
Found  Tony Allen
player was never an all star
passed year
Found  Lance Allred
player was never an all star
passed year
Found  Darrell Allums
player was never an all star
passed year
Found  Morris Almond
player was never an all star
passed year
Found  Derrick Alston
player was never an all star
passed year
Found  Rafer Alston
player was never an all star
passed year
Found  Peter Aluma
player was never an all star
passed year
Found  John Amaechi
player was never an all star
passed year
Found  Ashraf Amaya
player was never an all star
passed year
Found  Al-Farouq Aminu
player was never an all star
passed year
Found  Lou Amundson
player was never an all star
passed year
Found  Chris Andersen
player was never an all star
passed year
Found  David Andersen
player was never an all star
passed year
Found  Alan Anderson
player was never an all star
passed year
Found  Antonio Anderson
player was never an all star
passed year
Found  Derek Anderson
player was never an all star
passed y

passed year
Found  Kostas Antetokounmpo
player was never an all star
passed year
Found  Thanasis Antetokounmpo
player was never an all star
passed year
Found  Carmelo Anthony
player was an all star

 YEAR DATA: 
 [<a href="/players/a/anthoca01/gamelog/2004/">2003-04</a>, <a href="/teams/DEN/2004.html">DEN</a>, <a href="/leagues/NBA_2004.html">NBA</a>] 

2003-04
valid year
0
not an all star this year, 2003-04

 YEAR DATA: 
 [<a href="/players/a/anthoca01/gamelog/2005/">2004-05</a>, <a href="/teams/DEN/2005.html">DEN</a>, <a href="/leagues/NBA_2005.html">NBA</a>] 

2004-05
valid year
1
not an all star this year, 2004-05

 YEAR DATA: 
 [<a href="/players/a/anthoca01/gamelog/2006/">2005-06</a>, <a href="/teams/DEN/2006.html">DEN</a>, <a href="/leagues/NBA_2006.html">NBA</a>] 

2005-06
valid year
2
not an all star this year, 2005-06

 YEAR DATA: 
 [<a href="/players/a/anthoca01/gamelog/2007/">2006-07</a>, <a href="/teams/DEN/2007.html">DEN</a>, <a href="/leagues/NBA_2007.html">NBA</a>] 

20

passed year
Found  Trevor Ariza
player was never an all star
passed year
Found  Joe Arlauckas
player was never an all star
passed year
Found  B.J. Armstrong
player was an all star

 YEAR DATA: 
 [<a href="/players/a/armstbj01/gamelog/1990/">1989-90</a>, <a href="/teams/CHI/1990.html">CHI</a>, <a href="/leagues/NBA_1990.html">NBA</a>] 

1989-90
valid year
0
not an all star this year, 1989-90

 YEAR DATA: 
 [<a href="/players/a/armstbj01/gamelog/1991/">1990-91</a>, <a href="/teams/CHI/1991.html">CHI</a>, <a href="/leagues/NBA_1991.html">NBA</a>] 

1990-91
valid year
1
not an all star this year, 1990-91

 YEAR DATA: 
 [<a href="/players/a/armstbj01/gamelog/1992/">1991-92</a>, <a href="/teams/CHI/1992.html">CHI</a>, <a href="/leagues/NBA_1992.html">NBA</a>] 

1991-92
valid year
2
not an all star this year, 1991-92

 YEAR DATA: 
 [<a href="/players/a/armstbj01/gamelog/1993/">1992-93</a>, <a href="/teams/CHI/1993.html">CHI</a>, <a href="/leagues/NBA_1993.html">NBA</a>] 

1992-93
valid year
3

In [14]:
df_players_a = df_players
df_players_a.to_csv('Data/player-stats-a')

In [33]:
# Grab player data for B
df_players_b =  pd.DataFrame()
df_players_b, names_b, count_b = player_scrape('b', df_players_b, delay=2)

first run

Players career during/after 1979
Found  Chris Babb
player was never an all star

Players career during/after 1979
Found  Luke Babbitt
player was never an all star

Players career during/after 1979
Found  Miloš Babić
player was never an all star

Players career during/after 1979
Found  Dwayne Bacon
player was never an all star

Players career during/after 1979
Found  Dalibor Bagarić
player was never an all star

Players career during/after 1979
Found  John Bagley
player was never an all star

Players career during/after 1979
Found  Marvin Bagley
player was never an all star

Players career during/after 1979
Found  Carl Bailey
player was never an all star

Players career during/after 1979
Found  Gus Bailey
player was never an all star

Players career during/after 1979
Found  James Bailey
player was never an all star

Players career during/after 1979
Found  Thurl Bailey
player was never an all star

Players career during/after 1979
Found  Toby Bailey
player was never an all sta


Players career during/after 1979
Found  Mike Batiste
player was never an all star

Players career during/after 1979
Found  Tony Battie
player was never an all star

Players career during/after 1979
Found  Shane Battier
player was never an all star

Players career during/after 1979
Found  John Battle
player was never an all star

Players career during/after 1979
Found  Kenny Battle
player was never an all star

Players career during/after 1979
Found  Dave Batton
player was never an all star

Players career during/after 1979
Found  Nicolas Batum
player was never an all star

Players career during/after 1979
Found  Lonny Baxter
player was never an all star
Blocked


In [50]:
# attempt to loop through multiple letters
df_players_mult = pd.DataFrame()
letter_finished = False
delay = 0.5
for letter_iter in alphabet[13:]:
    # Reset state variables
    print(letter_iter)
    count_mult = 0
    letter_finished = False
    while not letter_finished:
        if count_mult == 0:
            df_players_mult, names_mult, count_mult, letter_finished = player_scrape(letter_iter, df_players_mult, delay=delay)
        else:
            df_players_mult, names_mult, count_mult, letter_finished = player_scrape(letter_iter, df_players_mult, delay=delay, first_run=0, names=names_mult, count=count_mult)


n
first run

Players career during/after 1979
Found  Hamady N'Diaye
player was never an all star

Players career during/after 1979
Found  Makhtar N'Diaye
player was never an all star

Players career during/after 1979
Found  Mamadou N'Diaye
player was never an all star

Players career during/after 1979
Found  Boniface N'Dong
player was never an all star

Players career during/after 1979
Found  Boštjan Nachbar
player was never an all star

Players career during/after 1979
Found  Abdel Nader
player was never an all star

Players career during/after 1979
Found  Lee Nailon
player was never an all star

Players career during/after 1979
Found  Eduardo Nájera
player was never an all star

Players career during/after 1979
Found  Larry Nance
player was an all star
1981-82
valid year
0
not an all star this year, 1981-82
1982-83
valid year
1
not an all star this year, 1982-83
1983-84
valid year
2
not an all star this year, 1983-84
1984-85
valid year
3
all star this year,  1984-85
1985-86
valid yea


Players career during/after 1979
Found  Norm Nixon
player was an all star
1977-78
valid year
0
not an all star this year, 1977-78
1978-79
valid year
1
not an all star this year, 1978-79
1979-80
valid year
2
not an all star this year, 1979-80
1980-81
valid year
3
not an all star this year, 1980-81
1981-82
valid year
4
all star this year,  1981-82
1982-83
valid year
5
not an all star this year, 1982-83
1983-84
valid year
6
not an all star this year, 1983-84
1984-85
valid year
7
all star this year,  1984-85
1985-86
valid year
8
not an all star this year, 1985-86
1988-89
valid year
11
not an all star this year, 1988-89

Players career during/after 1979
Found  Zeke Nnaji
player was never an all star

Players career during/after 1979
Found  Joakim Noah
player was an all star
2007-08
valid year
0
not an all star this year, 2007-08
2008-09
valid year
1
not an all star this year, 2008-09
2009-10
valid year
2
not an all star this year, 2009-10
2010-11
valid year
3
not an all star this year, 201


Players career during/after 1979
Found  Fabricio Oberto
player was never an all star

Players career during/after 1979
Found  Daniel Ochefu
player was never an all star

Players career during/after 1979
Found  Greg Oden
player was never an all star

Players career during/after 1979
Found  Lamar Odom
player was never an all star

Players career during/after 1979
Found  Alan Ogg
player was never an all star

Players career during/after 1979
Found  Tim Ohlbrecht
player was never an all star

Players career during/after 1979
Found  Semi Ojeleye
player was never an all star

Players career during/after 1979
Found  Emeka Okafor
player was never an all star

Players career during/after 1979
Found  Jahlil Okafor
player was never an all star

Players career during/after 1979
Found  Chuma Okeke
player was never an all star

Players career during/after 1979
Found  Elie Okobo
player was never an all star

Players career during/after 1979
Found  Josh Okogie
player was never an all star

Players ca


Players career during/after 1979
Found  Anthony Parker
player was never an all star

Players career during/after 1979
Found  Jabari Parker
player was never an all star

Players career during/after 1979
Found  Smush Parker
player was never an all star

Players career during/after 1979
Found  Sonny Parker
player was never an all star

Players career during/after 1979
Found  Tony Parker
player was an all star
2001-02
valid year
0
not an all star this year, 2001-02
2002-03
valid year
1
not an all star this year, 2002-03
2003-04
valid year
2
not an all star this year, 2003-04
2004-05
valid year
3
not an all star this year, 2004-05
2005-06
valid year
4
all star this year,  2005-06
2006-07
valid year
5
all star this year,  2006-07
2007-08
valid year
6
not an all star this year, 2007-08
2008-09
valid year
7
all star this year,  2008-09
2009-10
valid year
8
not an all star this year, 2009-10
2010-11
valid year
9
not an all star this year, 2010-11
2011-12
valid year
10
all star this year,  2011


Players career during/after 1979
Found  Reggie Perry
player was never an all star

Players career during/after 1979
Found  Tim Perry
player was never an all star

Players career during/after 1979
Found  Chuck Person
player was never an all star

Players career during/after 1979
Found  Wesley Person
player was never an all star

Players career during/after 1979
Found  Alec Peters
player was never an all star

Players career during/after 1979
Found  Jim Petersen
player was never an all star

Players career during/after 1979
Found  Morris Peterson
player was never an all star

Players career during/after 1979
Found  Johan Petro
player was never an all star

Players career during/after 1979
Found  Dražen Petrović
player was never an all star

Players career during/after 1979
Found  Richard Petruška
player was never an all star

Players career during/after 1979
Found  Roger Phegley
player was never an all star

Players career during/after 1979
Found  Derrick Phelps
player was never an all 

...


Players career during/after 1979
Found  Samaki Walker
player was never an all star

Players career during/after 1979
Found  Wally Walker
player was never an all star

Players career during/after 1979
Found  John Wall
player was an all star
2010-11
valid year
0
not an all star this year, 2010-11
2011-12
valid year
1
not an all star this year, 2011-12
2012-13
valid year
2
not an all star this year, 2012-13
2013-14
valid year
3
all star this year,  2013-14
2014-15
valid year
4
all star this year,  2014-15
2015-16
valid year
5
all star this year,  2015-16
2016-17
valid year
6
all star this year,  2016-17
2017-18
valid year
7
all star this year,  2017-18
2018-19
valid year
8
not an all star this year, 2018-19
2020-21
valid year
10
not an all star this year, 2020-21

Players career during/after 1979
Found  Ben Wallace
player was an all star
1996-97
valid year
0
not an all star this year, 1996-97
1997-98
valid year
1
not an all star this year, 1997-98
1998-99
valid year
2
not an all star thi


Players career during/after 1979
Found  Travis Wear
player was never an all star

Players career during/after 1979
Found  Clarence Weatherspoon
player was never an all star

Players career during/after 1979
Found  Nick Weatherspoon
player was never an all star

Players career during/after 1979
Found  Quinndary Weatherspoon
player was never an all star

Players career during/after 1979
Found  Kyle Weaver
player was never an all star

Players career during/after 1979
Found  James Webb
player was never an all star

Players career during/after 1979
Found  Marcus Webb
player was never an all star

Players career during/after 1979
Found  Spud Webb
player was never an all star

Players career during/after 1979
Found  Chris Webber
player was an all star
1993-94
valid year
0
not an all star this year, 1993-94
1994-95
valid year
1
not an all star this year, 1994-95
1995-96
valid year
2
not an all star this year, 1995-96
1996-97
valid year
3
all star this year,  1996-97
1997-98
valid year
4
not 


Players career during/after 1979
Found  Okaro White
player was never an all star

Players career during/after 1979
Found  Randy White
player was never an all star

Players career during/after 1979
Found  Rodney White
player was never an all star

Players career during/after 1979
Found  Rory White
player was never an all star

Players career during/after 1979
Found  Royce White
player was never an all star

Players career during/after 1979
Found  Rudy White
player was never an all star

Players career during/after 1979
Found  Tony White
player was never an all star

Players career during/after 1979
Found  Willie White
player was never an all star

Players career during/after 1979
Found  Isaiah Whitehead
player was never an all star

Players career during/after 1979
Found  Jerome Whitehead
player was never an all star

Players career during/after 1979
Found  Donald Whiteside
player was never an all star

Players career during/after 1979
Found  Hassan Whiteside
player was never an all st


Players career during/after 1979
Found  Guy Williams
player was never an all star

Players career during/after 1979
Found  Herb Williams
player was never an all star

Players career during/after 1979
Found  Hot Rod Williams
player was never an all star

Players career during/after 1979
Found  Jason Williams
player was never an all star

Players career during/after 1979
Found  Jawad Williams
player was never an all star

Players career during/after 1979
Found  Jay Williams
player was never an all star

Players career during/after 1979
Found  Jayson Williams
player was an all star
1990-91
valid year
0
not an all star this year, 1990-91
1991-92
valid year
1
not an all star this year, 1991-92
1992-93
valid year
2
not an all star this year, 1992-93
1993-94
valid year
3
not an all star this year, 1993-94
1994-95
valid year
4
not an all star this year, 1994-95
1995-96
valid year
5
not an all star this year, 1995-96
1996-97
valid year
6
not an all star this year, 1996-97
1997-98
valid year
7



Players career during/after 1979
Found  Kennard Winchester
player was never an all star

Players career during/after 1979
Found  Dylan Windler
player was never an all star

Players career during/after 1979
Found  David Wingate
player was never an all star

Players career during/after 1979
Found  Dontonio Wingfield
player was never an all star

Players career during/after 1979
Found  Justise Winslow
player was never an all star

Players career during/after 1979
Found  Rickie Winslow
player was never an all star

Players career during/after 1979
Found  Cassius Winston
player was never an all star

Players career during/after 1979
Found  Trevor Winter
player was never an all star

Players career during/after 1979
Found  Brian Winters
player was an all star
1974-75
valid year
0
not an all star this year, 1974-75
1975-76
valid year
1
all star this year,  1975-76
1976-77
valid year
2
not an all star this year, 1976-77
1977-78
valid year
3
all star this year,  1977-78
1978-79
valid year
4
no

In [62]:
df_players_mult.tail(15)

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Name,All Star
0,2005-06,24.0,NJN,NBA,PG,2,0,16,1,1.5,...,1.5,2,3.5,0,0,2,2,2,Derrick Zimmerman,0.0
0,2016-17,20.0,ORL,NBA,C,19,0,5.7,0.5,1.6,...,1.3,1.8,0.2,0.1,0.3,0.2,0.9,1.2,Stephen Zimmerman,0.0
0,2016-17,22.0,CHI,NBA,SF,44,18,19.2,2,5,...,2.5,2.8,0.8,0.3,0.4,0.9,1.8,5.5,Paul Zipser,0.0
1,2017-18,23.0,CHI,NBA,SF,54,12,15.3,1.5,4.3,...,2.2,2.4,0.9,0.4,0.3,0.8,1.6,4,Paul Zipser,0.0
0,2017-18,21.0,CLE,NBA,C,32,2,6.7,1.5,2.1,...,1.1,1.9,0.2,0.1,0.4,0.3,0.9,3.7,Ante Žižić,0.0
1,2018-19,22.0,CLE,NBA,C,59,25,18.3,3.1,5.6,...,3.6,5.4,0.9,0.2,0.4,1,1.9,7.8,Ante Žižić,0.0
2,2019-20,23.0,CLE,NBA,C,22,0,10,1.9,3.3,...,2.2,3,0.3,0.3,0.2,0.5,1.2,4.4,Ante Žižić,0.0
0,1982-83,29.0,DET,NBA,C,7,0,4.3,0.1,0.7,...,0.7,1.1,0.1,0.1,0.4,0.6,1.3,0.3,Jim Zoet,0.0
0,2016-17,19.0,LAL,NBA,C,38,11,16,3.3,6.3,...,3.1,4.2,0.8,0.4,0.9,0.8,1.7,7.5,Ivica Zubac,0.0
1,2017-18,20.0,LAL,NBA,C,43,0,9.5,1.4,2.8,...,1.8,2.9,0.6,0.2,0.3,0.6,1.1,3.7,Ivica Zubac,0.0


In [63]:
df_players_mult.to_csv('Data/Player-stats/player-stats-n-z')

In [ ]:
# merge tables together
# remember to drop one of the batches of players with n